In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, classification_report
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.ensemble import RandomForestClassifier


sns.set_style("darkgrid")
sns.color_palette()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Importing Dataset
teldata = pd.read_csv('../input/telecom-churn/telecom_churn_data.csv')

In [ ]:
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)
teldata.head()

In [ ]:
teldata.shape

In [ ]:
teldata.describe(include='all')

In [ ]:
missingdata = pd.DataFrame(teldata.isnull().sum()*100/teldata.shape[0])
missingdata.reset_index(inplace=True)
missingdata.rename(columns={'index':'Feature',0:'Missing%'}, inplace=True)

In [ ]:
missingdata.sort_values(by='Missing%', ascending=False)

In [ ]:
missingdata[missingdata['Missing%']>0].count()

In [ ]:
rech_columns = teldata.columns[teldata.columns.str.contains('rech_data|rech_amt')]
rech_columns

In [ ]:
teldata.loc[:, rech_columns].describe()

In [ ]:
#Imputing missing values/NaNs with 0s
teldata[rech_columns] = teldata[rech_columns].apply(lambda x: x.fillna(0)) 

In [ ]:
missingdata = pd.DataFrame(teldata.isnull().sum()*100/teldata.shape[0])
missingdata.reset_index(inplace=True)
missingdata.rename(columns={'index':'Feature',0:'Missing%'}, inplace=True)

In [ ]:
missingdata[missingdata['Missing%']>0].count()

In [ ]:
# Dropping columns with missing values > 30% threshold
cols_to_drop = missingdata[missingdata['Missing%'] >=30]
cols_to_drop

In [ ]:
cols_drop = cols_to_drop.Feature.values
cols_drop

In [ ]:
teldata.drop(columns= cols_drop, axis=1, inplace=True)

In [ ]:
final_missingdata = pd.DataFrame(teldata.isnull().sum()*100/teldata.shape[0])
final_missingdata.reset_index(inplace=True)
final_missingdata.rename(columns={'index':'Feature',0:'Missing%'}, inplace=True)
final_missingdata.sort_values(by='Missing%', ascending=False)

In [ ]:
teldata.shape

In [ ]:
# Dropping unwanted columns
date_cols = ['circle_id','last_date_of_month_6','last_date_of_month_7','last_date_of_month_8','last_date_of_month_9','date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9','date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8','date_of_last_rech_data_9']
teldata.drop(date_cols, axis=1, inplace=True)

In [ ]:
final_missingdata = pd.DataFrame(teldata.isnull().sum()*100/teldata.shape[0])
final_missingdata.reset_index(inplace=True)
final_missingdata.rename(columns={'index':'Feature',0:'Missing%'}, inplace=True)
final_missingdata.sort_values(by='Missing%', ascending=False)

In [ ]:
teldata.shape

In [ ]:
teldata.head()

In [ ]:
# Imputing remaining missing values with median values of their respective columns
remaining_missing_Values_cols = final_missingdata[final_missingdata['Missing%']>0].Feature.values
for i in remaining_missing_Values_cols:
    teldata[i] = teldata[i].fillna(teldata[i].median())

In [ ]:
teldata.head()

In [ ]:
final_missingdata = pd.DataFrame(teldata.isnull().sum()*100/teldata.shape[0])
final_missingdata.reset_index(inplace=True)
final_missingdata.rename(columns={'index':'Feature',0:'Missing%'}, inplace=True)
final_missingdata.sort_values(by='Missing%', ascending=False)

In [ ]:
teldata.info(verbose=1)

In [ ]:
teldata.shape

In [ ]:
#calculating total data recharge amount
teldata['total_data_rech_6'] = teldata.total_rech_data_6 * teldata.av_rech_amt_data_6
teldata['total_data_rech_7'] = teldata.total_rech_data_7 * teldata.av_rech_amt_data_7

In [ ]:
# adding 'total_data_rech' + 'total_rech_amt'
teldata['amt_data_6'] = teldata.total_data_rech_6 + teldata.total_rech_amt_6
teldata['amt_data_7'] = teldata.total_data_rech_7 + teldata.total_rech_amt_7

In [ ]:
# calculating average amount of june and july
teldata['av_amt_data_6_7']=(teldata.amt_data_6 + teldata.amt_data_7)/2

In [ ]:
# finding the 70th Percentile of avg recharge amount of june/july('good phase')
high_value = teldata.av_amt_data_6_7.quantile(0.70)
print("The Value of recharge amount at 70th percentile: ", high_value)

In [ ]:
highvaldata = teldata.loc[teldata.av_amt_data_6_7 >= high_value,:]
highvaldata = highvaldata.reset_index(drop=True)
highvaldata.shape

In [ ]:
#drop features created for arriving at high value customers
highvaldata = highvaldata.drop(columns=['total_data_rech_6','total_data_rech_7','amt_data_6','amt_dat','av_amt_data_6_7'], axis=1)
highvaldata.shape

In [ ]:
# computing incoming and outgoing usage
highvaldata['total_ic_og_mou_9'] = highvaldata.total_ic_mou_9 + highvaldata.total_og_mou_9

#computing 2g & 3g data usage
highvaldata['total_2g_3g_mb_9'] = highvaldata.vol_2g_mb_9 + highvaldata.vol_3g_mb_9

In [ ]:
# creating 'churn' variable: Where the customers who have not used any data and calling services in september are said to be churned
highvaldata['churn'] = highvaldata.apply(lambda x: 1 if(x.total_ic_og_mou_9==0 and x.total_2g_3g_mb_9==0) else 0, axis=1)

In [ ]:
# drop computed columns
highvaldata.drop(columns=['total_ic_og_mou_9','total_2g_3g_mb_9'], axis=1, inplace=True)

In [ ]:
highvaldata.shape

In [ ]:
# Capping the outliers for all numeric columns to cap for >Q1 &<Q3 , where Q1 & Q3 are top & bottom 1%.
numeric_columns = highvaldata.iloc[:,2:].columns
for i in numeric_columns:
    Q1 = highvaldata[i].quantile(0.01)
    Q3= highvaldata[i].quantile(0.99)
    IQR = Q3-Q1
    highvaldata = highvaldata.loc[(highvaldata[i]>= Q1-1.5*IQR) & (highvaldata[i]<=Q3+1.5*IQR)]

In [ ]:
highvaldata.shape

In [ ]:
# Inspecting Churn ratio
churn_ratio = pd.DataFrame(highvaldata.churn.value_counts(normalize=True)*100)
churn_ratio['churn_value']=[0,1]
sns.barplot(x=churn_ratio['churn_value'], y=churn_ratio.churn)
plt.show()

In [ ]:
churn_ratio

In [ ]:
# Dropping columns related to 9th month after tagging churners
sept_columns = highvaldata.columns[highvaldata.columns.str.contains('_9')]
sept_columns

In [ ]:
highvaldata.drop(columns=sept_columns, axis=1, inplace=True)

In [ ]:
# dropping sep_vbc_3g column
highvaldata.drop(columns=['sep_vbc_3g'], axis=1, inplace=True)

In [ ]:
highvaldata.shape

In [ ]:
# data type conversions
highvaldata['mobile_number'] = highvaldata['mobile_number'].astype(object)
highvaldata['churn'] = highvaldata['churn'].astype(object)

In [ ]:
highvaldata.head()

In [ ]:
# Derived Variable - 1 : Deriving AON(Age on Network)/365 to get Customer tenurity interms of years.
highvaldata['aon_Years'] = highvaldata['aon'].apply(lambda x : round((x/365),1))

In [ ]:
# Derived Variable - 2: Total_loc_MOU(Minutes of Usage) (local_incoming_mou+local_outgoing_mou)
for i in range(6,9):
    highvaldata['Total_loc_mou_'+str(i)] = (highvaldata['loc_ic_mou_'+str(i)])+(highvaldata['loc_og_mou_'+str(i)])

In [ ]:
# Derived Variable - 3: Total_roam_MOU(Minutes of USage) ( roaming_incoming_mou+roaming_outgoing_mou)
for i in range(6,9):
    highvaldata['Total_roam_mou_'+str(i)] = (highvaldata['roam_ic_mou_'+str(i)])+(highvaldata['roam_og_mou_'+str(i)])

In [ ]:
highvaldata.head()

In [ ]:
highvaldata.info(verbose=1)

In [ ]:
def uni(col1, col2, title):
    plt.figure(figsize=[20,10])
    plt.subplot(2,1,1)
    plt.title(title+'\n', fontdict={'size':20})
    sns.distplot(highvaldata[col1])
    plt.subplot(2,1,2)
    sns.distplot(highvaldata[col2])
    plt.tight_layout()
    plt.show()

In [ ]:
uni('total_og_mou_6','total_ic_mou_6',"Outgoing Vs Incoming - 6")

In [ ]:
uni('total_og_mou_7','total_ic_mou_7',"Outgoing Vs Incoming - 7")

In [ ]:
uni('total_og_mou_8','total_ic_mou_8',"Outgoing Vs Incoming - 8")

In [ ]:
uni('total_rech_amt_6','max_rech_data_6',"Calls Recharge Vs Internet Recharge - 6")

In [ ]:
uni('total_rech_amt_7','max_rech_data_7',"Calls Recharge Vs Internet Recharge - 7")

In [ ]:
uni('total_rech_amt_8','max_rech_data_8',"Calls Recharge Vs Internet Recharge - 8")

In [ ]:
plt.figure(figsize=[20,10])
plt.subplot(3,1,1)
plt.title('Total_times_recharge - 6\n', fontdict={'size':20})
sns.distplot(highvaldata['total_rech_num_6'])
plt.subplot(3,1,2)
plt.title('Total_times_recharge - 7\n', fontdict={'size':20})
sns.distplot(highvaldata['total_rech_num_7'])
plt.subplot(3,1,3)
plt.title('Total_times_recharge - 8\n', fontdict={'size':20})
sns.distplot(highvaldata['total_rech_num_8'])
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=[20,10])
plt.subplot(3,1,1)
plt.title('Average_Recharge_Amount - 6\n', fontdict={'size':20})
sns.distplot(highvaldata['av_rech_amt_data_6'])
plt.subplot(3,1,2)
plt.title('Average_Recharge_Amount - 7\n', fontdict={'size':20})
sns.distplot(highvaldata['av_rech_amt_data_7'])
plt.subplot(3,1,3)
plt.title('Average_Recharge_Amount - 8\n', fontdict={'size':20})
sns.distplot(highvaldata['av_rech_amt_data_8'])
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=[20,10])
plt.title('Tenurity of Customer in the Existing Network\n', fontdict={'size':20})
sns.distplot(highvaldata['aon_Years'])
plt.xticks(size=15)
plt.tight_layout()
plt.show()

In [ ]:
def bivariate(colx,col6,col7,col8,title):
    plt.figure(figsize=[12,6])
    
    plt.subplot(1,3,1)
    plt.suptitle(title+'\n', fontsize=20)
    gplot1 = sns.barplot(x=highvaldata[colx], y=highvaldata[col6])
    for p in gplot1.patches:
        gplot1.annotate(round((p.get_height()),2),#text that you want to write on top
        (p.get_x() + p.get_width() / 2., p.get_height()),# coordinates where text should be there
        ha = 'center', va = 'center',size=12,# alignment of the text
        xytext = (0, 10),# distance of text from top of the patch
        textcoords = 'offset points'), #do not change and remove it
    
    
    
    
    plt.subplot(1,3,2)
    gplot2 = sns.barplot(x=highvaldata[colx], y=highvaldata[col7])
    for p in gplot2.patches:
        gplot2.annotate(round((p.get_height()),2),#text that you want to write on top
        (p.get_x() + p.get_width() / 2., p.get_height()),# coordinates where text should be there
        ha = 'center', va = 'center',size=12,# alignment of the text
        xytext = (0, 10),# distance of text from top of the patch
        textcoords = 'offset points'), #do not change and remove it
    
    
    plt.subplot(1,3,3)
    gplot3 = sns.barplot(x=highvaldata[colx], y=highvaldata[col8])
    for p in gplot3.patches:
        gplot3.annotate(round((p.get_height()),2),#text that you want to write on top
        (p.get_x() + p.get_width() / 2., p.get_height()),# coordinates where text should be there
        ha = 'center', va = 'center',size=12,# alignment of the text
        xytext = (0, 10),# distance of text from top of the patch
        textcoords = 'offset points'), #do not change and remove it
    


    
    
    plt.tight_layout()
    plt.show()

In [ ]:
bivariate('churn', 'total_ic_mou_6','total_ic_mou_7','total_ic_mou_8','Total Incoming calls - (6,7,8) Vs Churn')

In [ ]:
bivariate('churn', 'total_og_mou_6','total_og_mou_7','total_og_mou_8','Total Outgoing calls - (6,7,8) Vs Churn')

In [ ]:
bivariate('churn', 'total_rech_num_6','total_rech_num_7','total_rech_num_8','Total Times Recharged - (6,7,8) Vs Churn')

In [ ]:
bivariate('churn', 'total_rech_amt_6','total_rech_amt_7','total_rech_amt_8','Total Recharge(Calls) Amount - (6,7,8) Vs Churn')

In [ ]:
bivariate('churn', 'av_rech_amt_data_6','av_rech_amt_data_7','av_rech_amt_data_8','Average Internet Recharge Amount - (6,7,8) Vs Churn')